# **Data Generation MIMESIS**

In [ ]:
!pip install kafka-python -q
!pip install mimesis -q
!pip install fastavro -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.4/285.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 33.7 MB/s eta 0:00:00


In [ ]:
!pip install azure-eventhub -q
!pip install azure-eventhub-kafka -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 12.4 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement azure-eventhub-kafka (from versions: none)
ERROR: No matching distribution found for azure-eventhub-kafka


In [ ]:
!pip install azure-eventhub-checkpointstoreblob -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.6/353.6 kB 8.9 MB/s eta 0:00:00


In [ ]:
from azure.eventhub import EventHubProducerClient, EventData
from azure.eventhub.aio import EventHubProducerClient as EventHubProducerClientAsync
from azure.eventhub.extensions.checkpointstoreblob import BlobCheckpointStore
from mimesis import Person, Address, Datetime
import uuid, json, random, time, fastavro, io, datetime

with open('/content/ride_request_schema.json') as f:
    ride_schema = json.load(f)

EVENT_HUB_NAMESPACE = "iesstsabdbaa-grp-05-08"
EVENT_HUB_NAME = "ride_request_group7"
CONNECTION_STRING = "Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc="

producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STRING, eventhub_name=EVENT_HUB_NAME)

def generate_ride_request():
    address = Address()
    return {
        "request_id": str(uuid.uuid4()),
        "passenger_id": str(uuid.uuid4()),
        "pickup_lat": float(address.latitude()),
        "pickup_lng": float(address.longitude()),
        "destination_lat": float(address.latitude()),
        "destination_lng": float(address.longitude()),
        "ride_type": random.choice(["standard", "premium", "pool"]),
        "status": random.choice(["requested", "cancelled", "accepted", "completed"]),
        "estimate_price": round(random.uniform(5, 50), 2),
        "demand_level": random.choice(["high", "low"]),
        "estimated_distance_km": round(random.uniform(1, 20), 2),
        "datetime": datetime.datetime.utcnow().isoformat()}

print(f"Producing ride requests to {EVENT_HUB_NAME}...")
try:
    while True:
        data = generate_ride_request()
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(json.dumps(data)))
        producer.send_batch(event_data_batch)
        print(f"Sent: {data['request_id']} at {data['datetime']}")
        time.sleep(random.uniform(0.1, 0.2))
except KeyboardInterrupt:
    print("Stopping producer...")
finally:
    producer.close()

Streaming output truncated to the last 5000 lines.
Sent: 211099de-dde5-47d1-af9f-e5666dd2d9f9 at 2025-04-21T10:32:35.653780
Sent: eb4ad5ac-500c-4b03-aca1-219de7384929 at 2025-04-21T10:32:35.946909
Sent: f08762f7-1c34-48d9-a7ed-ea63656baae4 at 2025-04-21T10:32:36.272079
Sent: f7361134-f0f3-41f0-847e-aeefb05b26f2 at 2025-04-21T10:32:36.588078
Sent: 6b629623-4d4b-4c5a-937d-70e12e45044d at 2025-04-21T10:32:36.899411
Sent: 19553ad2-d987-4bd9-839e-2f6040414777 at 2025-04-21T10:32:37.286587
Sent: 8148fb0d-1115-4e40-bb3a-8d8868301a3b at 2025-04-21T10:32:37.610317
Sent: aa6df09a-6b52-4e6a-b271-bf6d0546bcf9 at 2025-04-21T10:32:37.913831
Sent: 5a6344fe-2b18-4790-9b54-8885a95c28cd at 2025-04-21T10:32:38.206328
Sent: b9ca9901-9ec0-4722-8884-46ea88fa793b at 2025-04-21T10:32:38.507060
Sent: f7b02d99-7891-412e-9118-ad47de0f5a52 at 2025-04-21T10:32:38.863892
Sent: d2b0e25a-cae5-4a62-8b72-2beaa1b949d4 at 2025-04-21T10:32:39.204264
Sent: 4d8baf94-e7d4-49a0-89b1-55acf02b2a35 at 2025-04-21T10:32:39.517475


In [ ]:
with open('/content/driver_status_schema.json') as f:
    ride_schema = json.load(f)

EVENT_HUB_NAMESPACE = "iesstsabdbaa-grp-05-08"
EVENT_HUB_NAME = "driver_status_group7"
CONNECTION_STRING = "Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc="

producer = EventHubProducerClient.from_connection_string(conn_str=CONNECTION_STRING, eventhub_name=EVENT_HUB_NAME)

def generate_driver_status():
    address = Address()
    return {
        "driver_id": str(uuid.uuid4()),
        "current_lat": float(address.latitude()),
        "current_lng": float(address.longitude()),
        "status": random.choice(["available", "en_route", "on_trip", "offline"]),
        "vehicle_type": random.choice(["sedan", "SUV", "motorbike", "uber", "lyft", "cabify"]),
        "timestamp": datetime.datetime.utcnow().isoformat()}

print(f"Producing status requests to {EVENT_HUB_NAME}...")
try:
    while True:
        data = generate_driver_status()
        event_data_batch = producer.create_batch()
        event_data_batch.add(EventData(json.dumps(data)))
        producer.send_batch(event_data_batch)
        print(f"Sent: {data['driver_id']} ({data['status']}) at {data['timestamp']}")
        time.sleep(random.uniform(0.1, 0.15))
except KeyboardInterrupt:
    print("Stopping producer...")
finally:
    producer.close()

Streaming output truncated to the last 5000 lines.
Sent: bf987fe6-d600-4c7f-bcec-231f9139ac91 (on_trip) at 2025-04-21T11:19:18.175628
Sent: 8d885fb7-54b6-4584-8e41-a47a0da464f3 (offline) at 2025-04-21T11:19:18.470045
Sent: 4407e91c-65a7-4b9c-993b-8a4b9a11375f (on_trip) at 2025-04-21T11:19:18.769865
Sent: 8298b738-107c-4fe0-9cda-6ca81f03bbe8 (on_trip) at 2025-04-21T11:19:19.078272
Sent: 5ecbe4c9-2a49-4848-94ed-11f8e708e99b (en_route) at 2025-04-21T11:19:19.343706
Sent: a030fa55-3377-4915-af21-5570ac25d6b4 (offline) at 2025-04-21T11:19:19.650631
Sent: 27ad7e66-9af6-4f94-b3f4-841c87727386 (available) at 2025-04-21T11:19:19.932483
Sent: 43c5e45f-5ce0-42c5-ace3-fc63215f2aa2 (on_trip) at 2025-04-21T11:19:20.223164
Sent: dde1626a-12a7-44cd-9344-0fe914a8d6b8 (offline) at 2025-04-21T11:19:20.519773
Sent: c68626c8-908b-45af-aacf-29bca16f8a36 (on_trip) at 2025-04-21T11:19:20.806801
Sent: 2b2c2a03-7ae1-48d0-9ec0-f602681f7bb3 (available) at 2025-04-21T11:19:21.106499
Sent: 486bdd39-adff-47c5-bde3-d

# **Spark**

In [ ]:
# Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Download Spark 3.4.1 with Hadoop 3
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Unpack Spark
!tar -xvzf spark-3.4.1-bin-hadoop3.tgz

# Install findspark
!pip install -q findspark


spark-3.4.1-bin-hadoop3/
spark-3.4.1-bin-hadoop3/R/
spark-3.4.1-bin-hadoop3/R/lib/
spark-3.4.1-bin-hadoop3/R/lib/sparkr.zip
spark-3.4.1-bin-hadoop3/R/lib/SparkR/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/R.css
spark-3.4.1-bin-hadoop3/R/lib/SparkR/html/00Index.html
spark-3.4.1-bin-hadoop3/R/lib/SparkR/INDEX
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/aliases.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/AnIndex
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdx
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/SparkR.rdb
spark-3.4.1-bin-hadoop3/R/lib/SparkR/help/paths.rds
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/worker.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/worker/daemon.R
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/
spark-3.4.1-bin-hadoop3/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.4.1-bin-hadoop3/R/lib/S

In [ ]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()



In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaSparkIntegration") \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1,"
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .getOrCreate()


In [ ]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, FloatType
import time
from threading import Thread, Event
import os

# ==============================================
# 1. INITIALIZATION AND CONFIGURATION
# ==============================================

# Blob Storage Credentials
spark.conf.set(
    "fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
    "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g=="
)

# Stream Optimization Settings
STREAM_CONFIG = {
    "processing_time": "10 seconds",
    "max_offsets": 5000,
    "min_offsets": 1000,
    "run_duration": 1200,  # 20 minutes
    "checkpoint_path": "wasbs://group7@iesstsabdbaa.blob.core.windows.net/new_checkpoints"
}

# Event Hub Configurations
event_hub_namespace = "iesstsabdbaa-grp-05-08"

ride_config = {
    "name": "ride_request_group7",
    "kafka_conf": {
        "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
        "subscribe": "ride_request_group7",
        "kafka.sasl.mechanism": "PLAIN",
        "kafka.security.protocol": "SASL_SSL",
        "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc=;EntityPath=ride_request_group7";'
    }
}

driver_config = {
    "name": "driver_status_group7",
    "kafka_conf": {
        "kafka.bootstrap.servers": f"{event_hub_namespace}.servicebus.windows.net:9093",
        "subscribe": "driver_status_group7",
        "kafka.sasl.mechanism": "PLAIN",
        "kafka.security.protocol": "SASL_SSL",
        "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://iesstsabdbaa-grp-05-08.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=5RG+c6g2Z/sBZL+ZwtHGb1rowkMZgljui+AEhJZ+NXc=;EntityPath=driver_status_group7";'
    }
}

# Define Schemas
ride_schema = StructType() \
    .add("request_id", StringType()) \
    .add("passenger_id", StringType()) \
    .add("pickup_lat", FloatType()) \
    .add("pickup_lng", FloatType()) \
    .add("destination_lat", FloatType()) \
    .add("destination_lng", FloatType()) \
    .add("ride_type", StringType()) \
    .add("status", StringType()) \
    .add("estimate_price", FloatType()) \
    .add("demand_level", StringType()) \
    .add("estimated_distance_km", FloatType()) \
    .add("datetime", StringType())

driver_schema = StructType() \
    .add("driver_id", StringType()) \
    .add("current_lat", FloatType()) \
    .add("current_lng", FloatType()) \
    .add("status", StringType()) \
    .add("vehicle_type", StringType()) \
    .add("timestamp", StringType())

# ==============================================
# 2. STREAM INITIALIZATION
# ==============================================

def init_optimized_stream(source_config, schema, stream_name):
    df = (spark.readStream
          .format("kafka")
          .options(**source_config["kafka_conf"])
          .option("startingOffsets", "earliest")
          .load()
          .selectExpr("CAST(value AS STRING) as json_str")
          .select(from_json(col("json_str"), schema).alias("data"))
          .select("data.*"))

    return (df.writeStream
            .format("parquet")
            .option("path", f"wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_{stream_name}_data")
            .option("checkpointLocation", f"{STREAM_CONFIG['checkpoint_path']}/{stream_name}")
            .trigger(processingTime=STREAM_CONFIG["processing_time"])
            .option("maxOffsetsPerTrigger", STREAM_CONFIG["max_offsets"])
            .option("minOffsetsPerTrigger", STREAM_CONFIG["min_offsets"])
            .outputMode("append")
            .queryName(f"optimized_{stream_name}"))

# Start both streams
ride_stream = init_optimized_stream(ride_config, ride_schema, "ride").start()
driver_stream = init_optimized_stream(driver_config, driver_schema, "driver").start()

# ==============================================
# 3. MONITORING SYSTEM
# ==============================================

def monitor():
    start = time.time()
    while time.time() - start < STREAM_CONFIG["run_duration"]:
        time.sleep(30)
        print("\n=== Stream Progress ===")
        for stream in [ride_stream, driver_stream]:
            status = stream.status
            progress = stream.lastProgress or {}
            print(f"\n{stream.name}:")
            print(f"Status: {status['message']}")
            print(f"Records processed: {progress.get('numInputRows', 0)}")
            print(f"Processing rate: {progress.get('inputRowsPerSecond', 0):.1f} rows/sec")
            print(f"Latest offsets: {progress.get('sources', [{}])[0].get('endOffset', 'N/A')}")

# Start monitoring thread
monitor_thread = Thread(target=monitor)
monitor_thread.daemon = True
monitor_thread.start()

# ==============================================
# 4. SHUTDOWN HANDLING
# ==============================================

try:
    monitor_thread.join(STREAM_CONFIG["run_duration"])
except KeyboardInterrupt:
    print("\nManual shutdown requested")

print("\n=== Stopping Streams ===")
ride_stream.stop()
driver_stream.stop()

# ==============================================
# 5. FINAL VERIFICATION
# ==============================================

print("\n=== Final Data Verification ===")
for name, path in [("Ride", "optimized_ride_data"), ("Driver", "optimized_driver_data")]:
    try:
        full_path = f"wasbs://group7@iesstsabdbaa.blob.core.windows.net/{path}/*"
        df = spark.read.parquet(full_path)
        count = df.count()
        print(f"\n{name} records: {count}")
        if count > 0:
            df.show(5, truncate=False)
        print(f"Storage location: {full_path}")
    except Exception as e:
        print(f"\nError reading {name} data: {str(e)}")

spark.stop()
print("\nProcessing complete!")


=== Stream Progress ===

optimized_ride:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

optimized_driver:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

=== Stream Progress ===

optimized_ride:
Status: Waiting for next trigger
Records processed: 5939
Processing rate: 0.0 rows/sec
Latest offsets: {'ride_request_group7': {'2': 1484, '1': 1485, '3': 1485, '0': 1485}}

optimized_driver:
Status: Processing new data
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: N/A

=== Stream Progress ===

optimized_ride:
Status: Waiting for next trigger
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: {'ride_request_group7': {'2': 1484, '1': 1485, '3': 1485, '0': 1485}}

optimized_driver:
Status: Waiting for next trigger
Records processed: 0
Processing rate: 0.0 rows/sec
Latest offsets: {'driver_status_group7': {'2': 1661, '1': 1661, '3': 1661, '0': 1663}}

=== S

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

# 1. Initialize Spark (if not already running)
spark = SparkSession.builder \
    .appName("RideAnalysis") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net", "your_access_key_here") \
    .getOrCreate()

# 2. Read stored data
def analyze_data():
    try:
        print("Reading ride data...")
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*")

        print("Reading driver data...")
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*")

        # 3. Perform analysis
        print("\n=== Ride Analysis ===")
        ride_df.groupBy("status").count().show()
        ride_df.groupBy("ride_type").avg("estimate_price").show()

        print("\n=== Driver Analysis ===")
        driver_df.groupBy("status").count().show()
        driver_df.groupBy("vehicle_type").count().show()

        # 4. Sample visualization data
        print("\nTop 5 Rides:")
        ride_df.show(5)

        print("\nTop 5 Drivers:")
        driver_df.show(5)

        return ride_df, driver_df

    except Exception as e:
        print(f"Analysis error: {str(e)}")
        return None, None

# Run analysis
ride_df, driver_df = analyze_data()

# 5. Keep Spark running for further queries
# spark.stop()  # Only run this when completely done

Reading ride data...
Reading driver data...

=== Ride Analysis ===
+---------+-----+
|   status|count|
+---------+-----+
|completed| 1534|
|cancelled| 1523|
|requested| 1442|
| accepted| 1440|
+---------+-----+

+---------+-------------------+
|ride_type|avg(estimate_price)|
+---------+-------------------+
|     pool|  27.51182896962451|
| standard| 27.578480985134462|
|  premium| 27.536398779295695|
+---------+-------------------+


=== Driver Analysis ===
+---------+-----+
|   status|count|
+---------+-----+
|  offline| 1690|
|  on_trip| 1626|
|available| 1620|
| en_route| 1710|
+---------+-----+

+------------+-----+
|vehicle_type|count|
+------------+-----+
|         SUV| 1092|
|        lyft| 1101|
|      cabify| 1141|
|   motorbike| 1148|
|       sedan| 1067|
|        uber| 1097|
+------------+-----+


Top 5 Rides:
+--------------------+--------------------+----------+----------+---------------+---------------+---------+---------+--------------+------------+---------------------+-

# **Dashboard example**
Redo it using streamlit

In [ ]:
# First install required packages
!pip install pyspark==3.4.1 pandas matplotlib ipywidgets
!pip install --upgrade pyspark[hadoop]

  Using cached pyspark-3.4.1-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.5
    Uninstalling pyspark-3.5.5:
      Successfully uninstalled pyspark-3.5.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-spark-connect 0.5.2 requires pyspark>=3.5, but you have pyspark 3.4.1 which is incompatible.
  Using cached pyspark-3.5.5-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.4.1
    Uninstalling pyspark-3.4.1:
      Successfully uninstalled pyspark-3.4.1


In [ ]:
# Initialize Spark with Azure WASB support
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AzureBlobDashboard") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

# Verify Azure connectivity
try:
    files = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem \
        .get(spark.sparkContext._jsc.hadoopConfiguration()) \
        .listStatus(spark.sparkContext._jvm.org.apache.hadoop.fs.Path(
            "wasbs://group7@iesstsabdbaa.blob.core.windows.net/"
        ))
    print("✅ Successfully connected to Azure Blob Storage")
    print(f"Found {len(files)} items in container")
except Exception as e:
    print(f"❌ Connection failed: {str(e)}")

# Dashboard Imports
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

# Create widgets
refresh_button = widgets.Button(description="Refresh Data", button_style='success')
status_dropdown = widgets.Dropdown(
    options=['All', 'requested', 'accepted', 'completed', 'cancelled'],
    description='Filter Status:',
    style={'description_width': 'initial'}
)
output = widgets.Output()

# Load data function with error handling
def load_data():
    try:
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*")
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*")
        return ride_df.toPandas(), driver_df.toPandas()
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# Update dashboard function
def update_dashboard(_):
    with output:
        clear_output(wait=True)
        print("🔄 Loading data...")

        try:
            ride_df, driver_df = load_data()

            if ride_df.empty or driver_df.empty:
                print("⚠️ No data found - check storage connection")
                return

            # Filter by status
            if status_dropdown.value != 'All':
                ride_df = ride_df[ride_df['status'] == status_dropdown.value]

            # Create dashboard layout
            display(widgets.HTML("<h2>🚖 Ride Sharing Analytics Dashboard</h2>"))

            # Metrics row
            metrics_html = f"""
            <div style="display: flex; justify-content: space-between; margin: 10px 0;">
                <div class="metric-card">📊 Total Rides: <b>{len(ride_df)}</b></div>
                <div class="metric-card">🚕 Active Drivers: <b>{len(driver_df[driver_df['status'] != 'offline'])}</b></div>
                <div class="metric-card">💰 Avg Price: <b>${ride_df['estimate_price'].mean():.2f}</b></div>
                <div class="metric-card">📅 Last Updated: <b>{time.strftime('%H:%M:%S')}</b></div>
            </div>
            """
            display(widgets.HTML(metrics_html))

            # Visualizations
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))

            # Plot 1: Ride Type Distribution
            ride_df['ride_type'].value_counts().plot(
                kind='pie',
                ax=axes[0],
                autopct='%1.1f%%',
                colors=['#4CAF50', '#2196F3', '#FFC107']
            )
            axes[0].set_title('Ride Types', pad=20)

            # Plot 2: Status Distribution
            status_counts = ride_df['status'].value_counts()
            status_counts.plot(
                kind='bar',
                ax=axes[1],
                color=['#4CAF50', '#2196F3', '#FFC107', '#F44336']
            )
            axes[1].set_title('Ride Status', pad=20)
            axes[1].tick_params(axis='x', rotation=45)

            # Plot 3: Vehicle Types
            driver_df['vehicle_type'].value_counts().plot(
                kind='bar',
                ax=axes[2],
                color=['#4CAF50', '#2196F3', '#FFC107', '#9C27B0']
            )
            axes[2].set_title('Driver Vehicles', pad=20)
            axes[2].tick_params(axis='x', rotation=45)

            plt.tight_layout()
            plt.show()

            # Data preview
            display(widgets.HTML("<h4>Sample Ride Data</h4>"))
            display(ride_df.head(3))

        except Exception as e:
            print(f"⚠️ Dashboard error: {str(e)}")

# Set up interactions
refresh_button.on_click(update_dashboard)
status_dropdown.observe(update_dashboard, names='value')

# Initial layout
dashboard = widgets.VBox([
    widgets.HBox([refresh_button, status_dropdown]),
    output
])

# Initial display
display(dashboard)
update_dashboard(None)

# Auto-refresh option (uncomment if needed)
# import threading
# def auto_refresh():
#     while True:
#         time.sleep(30)
#         refresh_button.click()
# threading.Thread(target=auto_refresh, daemon=True).start()

❌ Connection failed: Wrong FS: wasbs://group7@iesstsabdbaa.blob.core.windows.net/, expected: file:///


In [ ]:
# Initialize Spark with Azure WASB support
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder \
    .appName("EnhancedRideAnalytics") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

# Verify Azure connectivity
try:
    files = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem \
        .get(spark.sparkContext._jsc.hadoopConfiguration()) \
        .listStatus(spark.sparkContext._jvm.org.apache.hadoop.fs.Path(
            "wasbs://group7@iesstsabdbaa.blob.core.windows.net/"
        ))
    print("✅ Successfully connected to Azure Blob Storage")
except Exception as e:
    print(f"❌ Connection failed: {str(e)}")

# Dashboard Imports
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import time
import folium
from datetime import datetime

# Install folium if needed
try:
    from folium import plugins
except:
    !pip install folium
    from folium import plugins

# Create widgets
refresh_button = widgets.Button(description="Refresh Dashboard", button_style='success')
status_dropdown = widgets.Dropdown(
    options=['All', 'requested', 'accepted', 'completed', 'cancelled'],
    description='Ride Status:',
    style={'description_width': 'initial'}
)
time_window = widgets.Dropdown(
    options=['Last Hour', 'Today', 'Last 7 Days', 'All Time'],
    value='Today',
    description='Time Window:',
    style={'description_width': 'initial'}
)
analysis_tabs = widgets.Tab()
output = widgets.Output()

# Load data function with enhanced time filtering
def load_data():
    try:
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*").toPandas()
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*").toPandas()

        # Convert timestamps
        ride_df['datetime'] = pd.to_datetime(ride_df['datetime'])
        driver_df['timestamp'] = pd.to_datetime(driver_df['timestamp'])

        return ride_df, driver_df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# Enhanced KPI calculations
def calculate_kpis(ride_df, driver_df):
    kpis = {}

    # Basic Metrics
    kpis['total_rides'] = len(ride_df)
    kpis['active_drivers'] = len(driver_df[driver_df['status'] != 'offline'])
    kpis['avg_price'] = ride_df['estimate_price'].mean()

    # Revenue Analysis
    kpis['total_revenue'] = ride_df['estimate_price'].sum()
    kpis['revenue_per_ride'] = kpis['total_revenue'] / kpis['total_rides'] if kpis['total_rides'] > 0 else 0

    # Time-Based Metrics
    time_diff = (ride_df['datetime'].max() - ride_df['datetime'].min()).total_seconds() / 3600
    kpis['rides_per_hour'] = kpis['total_rides'] / max(1, time_diff)

    # Driver Utilization
    kpis['driver_utilization'] = (kpis['active_drivers'] / len(driver_df)) * 100 if len(driver_df) > 0 else 0

    # Cancellation Analysis
    kpis['cancellation_rate'] = (len(ride_df[ride_df['status'] == 'cancelled']) / kpis['total_rides']) * 100 if kpis['total_rides'] > 0 else 0

    return kpis

# Update dashboard function with all new features
def update_dashboard(_):
    with output:
        clear_output(wait=True)
        start_time = time.time()

        try:
            # Load data
            ride_df, driver_df = load_data()

            if ride_df.empty or driver_df.empty:
                print("⚠️ No data found - check storage connection")
                return

            # Apply filters
            current_time = datetime.now()
            time_filter = {
                'Last Hour': pd.Timedelta(hours=1),
                'Today': pd.Timedelta(days=1),
                'Last 7 Days': pd.Timedelta(days=7),
                'All Time': pd.Timedelta(days=3650)  # 10 years
            }[time_window.value]

            ride_df = ride_df[ride_df['datetime'] >= (current_time - time_filter)]
            if status_dropdown.value != 'All':
                ride_df = ride_df[ride_df['status'] == status_dropdown.value]

            # Calculate KPIs
            kpis = calculate_kpis(ride_df, driver_df)

            # ====================
            # Dashboard Layout
            # ====================

            # Header
            display(HTML("<h1 style='color: #1a73e8;'>🚖 Advanced Ride Analytics Dashboard</h1>"))
            display(HTML(f"<p>Last Updated: {current_time.strftime('%Y-%m-%d %H:%M:%S')} | Processing Time: {(time.time()-start_time):.2f}s</p>"))

            # ====================
            # KPI Metrics Cards
            # ====================
            metrics_html = f"""
            <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 10px; margin: 15px 0;">
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>💰 Revenue</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>${kpis['total_revenue']:,.2f}</p>
                    <p style='margin: 0; font-size: 14px;'>${kpis['revenue_per_ride']:.2f} per ride</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>📊 Rides</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['total_rides']:,}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['rides_per_hour']:.1f} rides/hour</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>🚕 Drivers</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['active_drivers']}/{len(driver_df)}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['driver_utilization']:.1f}% utilization</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>⚠️ Cancellations</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['cancellation_rate']:.1f}%</p>
                    <p style='margin: 0; font-size: 14px;'>Avg price: ${ride_df[ride_df['status']=='cancelled']['estimate_price'].mean():.2f}</p>
                </div>
            </div>
            """
            display(HTML(metrics_html))

            # ====================
            # Main Visualization Tabs
            # ====================
            tab_outputs = []

            # Tab 1: Time Analysis
            with widgets.Output() as tab1:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                # Hourly Demand
                hourly = ride_df.set_index('datetime').resample('H')['request_id'].count()
                hourly.plot(ax=ax1, title='Hourly Ride Demand', color='#1a73e8')
                ax1.set_ylabel("Number of Rides")
                ax1.grid(True, alpha=0.3)

                # Price Distribution
                ride_df['estimate_price'].plot(kind='box', ax=ax2, vert=False,
                                             patch_artist=True,
                                             boxprops=dict(facecolor='#1a73e8'))
                ax2.set_title('Ride Price Distribution')
                ax2.set_xlabel("Price ($)")
                ax2.grid(True, alpha=0.3)

                plt.tight_layout()
                plt.show()

                # Weekday Analysis
                if time_window.value in ['Last 7 Days', 'All Time']:
                    fig, ax = plt.subplots(figsize=(10, 4))
                    ride_df['weekday'] = ride_df['datetime'].dt.day_name()
                    ride_df['weekday'].value_counts().reindex([
                        'Monday', 'Tuesday', 'Wednesday',
                        'Thursday', 'Friday', 'Saturday', 'Sunday'
                    ]).plot(kind='bar', ax=ax, color='#1a73e8')
                    ax.set_title('Rides by Weekday')
                    ax.set_ylabel("Number of Rides")
                    plt.tight_layout()
                    plt.show()

            # Tab 2: Geospatial Analysis
            with widgets.Output() as tab2:
                # Create base map
                avg_lat = ride_df['pickup_lat'].mean()
                avg_lng = ride_df['pickup_lng'].mean()
                m = folium.Map(location=[avg_lat, avg_lng], zoom_start=12)

                # Add heatmap
                heat_data = [[row['pickup_lat'], row['pickup_lng']]
                            for _, row in ride_df.iterrows()]
                plugins.HeatMap(heat_data, radius=15).add_to(m)

                # Add cluster markers
                marker_cluster = plugins.MarkerCluster().add_to(m)
                for _, row in ride_df.sample(min(100, len(ride_df))).iterrows():
                    folium.Marker(
                        [row['pickup_lat'], row['pickup_lng']],
                        popup=f"${row['estimate_price']} {row['ride_type']}",
                        icon=folium.Icon(color='blue' if row['status']=='completed' else 'red')
                    ).add_to(marker_cluster)

                # Display map
                display(m)

                # Price by Area analysis
                ride_df['pickup_area'] = pd.cut(ride_df['pickup_lat'], bins=5, labels=[
                    'South', 'Mid-South', 'Central', 'Mid-North', 'North'
                ])
                area_prices = ride_df.groupby('pickup_area')['estimate_price'].mean()
                plt.figure(figsize=(10, 4))
                area_prices.plot(kind='bar', color='#1a73e8')
                plt.title('Average Price by Area')
                plt.ylabel("Price ($)")
                plt.tight_layout()
                plt.show()

            # Tab 3: Driver-Ride Analysis
            with widgets.Output() as tab3:
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                # Driver Status
                driver_df['status'].value_counts().plot(
                    kind='pie', ax=ax1, autopct='%1.1f%%',
                    colors=['#34a853', '#1a73e8', '#fbbc05', '#ea4335']
                )
                ax1.set_title('Driver Status Distribution')

                # Vehicle Type Analysis
                vehicle_analysis = driver_df.groupby('vehicle_type').agg({
                    'driver_id': 'count',
                    'current_lat': 'mean'  # Just for demo
                })
                vehicle_analysis['driver_id'].plot(
                    kind='bar', ax=ax2, color='#1a73e8'
                )
                ax2.set_title('Drivers by Vehicle Type')
                ax2.set_ylabel("Number of Drivers")

                plt.tight_layout()
                plt.show()

                # Driver-Ride Correlation
                if not ride_df.empty and not driver_df.empty:
                    plt.figure(figsize=(10, 4))
                    hourly_drivers = driver_df.set_index('timestamp').resample('H')['driver_id'].count()
                    hourly_rides = ride_df.set_index('datetime').resample('H')['request_id'].count()
                    pd.DataFrame({
                        'Drivers': hourly_drivers,
                        'Rides': hourly_rides
                    }).plot(title='Drivers vs Rides (Hourly)')
                    plt.ylabel("Count")
                    plt.tight_layout()
                    plt.show()

            # Create tabs
            analysis_tabs.children = [tab1, tab2, tab3]
            analysis_tabs.titles = ['Time Analysis', 'Geospatial', 'Driver Insights']
            display(analysis_tabs)

            # ====================
            # Data Preview Section
            # ====================
            display(HTML("<h3 style='margin-top: 20px;'>📋 Data Preview</h3>"))

            # Create tabs for data preview
            preview_tabs = widgets.Tab()
            with widgets.Output() as preview1:
                display(HTML("<h4>Ride Data Sample</h4>"))
                display(ride_df[['datetime', 'ride_type', 'status', 'estimate_price']].head(5))

            with widgets.Output() as preview2:
                display(HTML("<h4>Driver Data Sample</h4>"))
                display(driver_df[['timestamp', 'status', 'vehicle_type']].head(5))

            preview_tabs.children = [preview1, preview2]
            preview_tabs.titles = ['Rides', 'Drivers']
            display(preview_tabs)

        except Exception as e:
            print(f"⚠️ Dashboard error: {str(e)}")
            import traceback
            traceback.print_exc()

# Set up interactions
refresh_button.on_click(update_dashboard)
status_dropdown.observe(update_dashboard, names='value')
time_window.observe(update_dashboard, names='value')

# Dashboard layout
controls = widgets.HBox([refresh_button, status_dropdown, time_window])
dashboard = widgets.VBox([
    controls,
    output
])

# Initial display
display(dashboard)
update_dashboard(None)

# Auto-refresh option
import threading
def auto_refresh():
    while True:
        time.sleep(300)  # Refresh every 5 minutes
        refresh_button.click()

threading.Thread(target=auto_refresh, daemon=True).start()

❌ Connection failed: Wrong FS: wasbs://group7@iesstsabdbaa.blob.core.windows.net/, expected: file:///


In [ ]:
# Initialize Spark with Azure WASB support
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder \
    .appName("EnhancedRideAnalytics") \
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .config("fs.azure.account.key.iesstsabdbaa.blob.core.windows.net",
            "yfqMW8gf8u+M5pOW33Q5gtRTFBJQXStVK4K2rlCVVzxlrRG21Sh7MVj06uExoL86Npb7HWWgxYUe+ASthUr6/g==") \
    .config("fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("fs.azure.account.keyprovider.iesstsabdbaa.blob.core.windows.net",
            "org.apache.hadoop.fs.azure.SimpleKeyProvider") \
    .getOrCreate()

# Dashboard Imports
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import time
import folium
from datetime import datetime

# Install folium if needed
try:
    from folium import plugins
except:
    !pip install folium
    from folium import plugins

# Create widgets
refresh_button = widgets.Button(description="Refresh Dashboard", button_style='success')
status_dropdown = widgets.Dropdown(
    options=['All', 'requested', 'accepted', 'completed', 'cancelled'],
    description='Ride Status:',
    style={'description_width': 'initial'}
)
time_window = widgets.Dropdown(
    options=['Last Hour', 'Today', 'Last 7 Days', 'All Time'],
    value='Today',
    description='Time Window:',
    style={'description_width': 'initial'}
)

# Initialize tabs properly
output = widgets.Output()
analysis_tabs = widgets.Tab()
tab1_output = widgets.Output()
tab2_output = widgets.Output()
tab3_output = widgets.Output()
analysis_tabs.children = [tab1_output, tab2_output, tab3_output]
analysis_tabs.titles = ['Time Analysis', 'Geospatial', 'Driver Insights']

# Load data function with enhanced time filtering
def load_data():
    try:
        ride_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_ride_data/*").toPandas()
        driver_df = spark.read.parquet("wasbs://group7@iesstsabdbaa.blob.core.windows.net/optimized_driver_data/*").toPandas()

        # Convert timestamps
        ride_df['datetime'] = pd.to_datetime(ride_df['datetime'])
        driver_df['timestamp'] = pd.to_datetime(driver_df['timestamp'])

        return ride_df, driver_df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return pd.DataFrame(), pd.DataFrame()

# Enhanced KPI calculations
def calculate_kpis(ride_df, driver_df):
    kpis = {}

    # Basic Metrics
    kpis['total_rides'] = len(ride_df)
    kpis['active_drivers'] = len(driver_df[driver_df['status'] != 'offline'])
    kpis['avg_price'] = ride_df['estimate_price'].mean()

    # Revenue Analysis
    kpis['total_revenue'] = ride_df['estimate_price'].sum()
    kpis['revenue_per_ride'] = kpis['total_revenue'] / kpis['total_rides'] if kpis['total_rides'] > 0 else 0

    # Time-Based Metrics
    time_diff = (ride_df['datetime'].max() - ride_df['datetime'].min()).total_seconds() / 3600
    kpis['rides_per_hour'] = kpis['total_rides'] / max(1, time_diff)

    # Driver Utilization
    kpis['driver_utilization'] = (kpis['active_drivers'] / len(driver_df)) * 100 if len(driver_df) > 0 else 0

    # Cancellation Analysis
    kpis['cancellation_rate'] = (len(ride_df[ride_df['status'] == 'cancelled']) / kpis['total_rides']) * 100 if kpis['total_rides'] > 0 else 0

    return kpis

# Update dashboard function with all new features
def update_dashboard(_):
    with output:
        clear_output(wait=True)
        start_time = time.time()

        try:
            # Load data
            ride_df, driver_df = load_data()

            if ride_df.empty or driver_df.empty:
                print("⚠️ No data found - check storage connection")
                return

            # Apply filters
            current_time = datetime.now()
            time_filter = {
                'Last Hour': pd.Timedelta(hours=1),
                'Today': pd.Timedelta(days=1),
                'Last 7 Days': pd.Timedelta(days=7),
                'All Time': pd.Timedelta(days=3650)  # 10 years
            }[time_window.value]

            ride_df = ride_df[ride_df['datetime'] >= (current_time - time_filter)]
            if status_dropdown.value != 'All':
                ride_df = ride_df[ride_df['status'] == status_dropdown.value]

            # Calculate KPIs
            kpis = calculate_kpis(ride_df, driver_df)

            # ====================
            # Dashboard Layout
            # ====================

            # Header
            display(HTML("<h1 style='color: #1a73e8;'>🚖 Advanced Ride Analytics Dashboard</h1>"))
            display(HTML(f"<p>Last Updated: {current_time.strftime('%Y-%m-%d %H:%M:%S')} | Processing Time: {(time.time()-start_time):.2f}s</p>"))

            # ====================
            # KPI Metrics Cards
            # ====================
            metrics_html = f"""
            <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 10px; margin: 15px 0;">
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>💰 Revenue</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>${kpis['total_revenue']:,.2f}</p>
                    <p style='margin: 0; font-size: 14px;'>${kpis['revenue_per_ride']:.2f} per ride</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>📊 Rides</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['total_rides']:,}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['rides_per_hour']:.1f} rides/hour</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>🚕 Drivers</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['active_drivers']}/{len(driver_df)}</p>
                    <p style='margin: 0; font-size: 14px;'>{kpis['driver_utilization']:.1f}% utilization</p>
                </div>
                <div style="padding: 15px; background: #f8f9fa; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1)">
                    <h3 style='margin: 0; color: #1a73e8;'>⚠️ Cancellations</h3>
                    <p style='font-size: 24px; margin: 5px 0;'>{kpis['cancellation_rate']:.1f}%</p>
                    <p style='margin: 0; font-size: 14px;'>Avg price: ${ride_df[ride_df['status']=='cancelled']['estimate_price'].mean():.2f}</p>
                </div>
            </div>
            """
            display(HTML(metrics_html))

            # ====================
            # Main Visualization Tabs
            # ====================

            # Clear previous tab contents
            for tab in analysis_tabs.children:
                tab.clear_output()

            # Tab 1: Time Analysis
            with tab1_output:
                try:
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                    # Hourly Demand
                    hourly = ride_df.set_index('datetime').resample('H')['request_id'].count()
                    hourly.plot(ax=ax1, title='Hourly Ride Demand', color='#1a73e8')
                    ax1.set_ylabel("Number of Rides")
                    ax1.grid(True, alpha=0.3)

                    # Price Distribution
                    ride_df['estimate_price'].plot(kind='box', ax=ax2, vert=False,
                                                 patch_artist=True,
                                                 boxprops=dict(facecolor='#1a73e8'))
                    ax2.set_title('Ride Price Distribution')
                    ax2.set_xlabel("Price ($)")
                    ax2.grid(True, alpha=0.3)

                    plt.tight_layout()
                    plt.show()

                    # Weekday Analysis
                    if time_window.value in ['Last 7 Days', 'All Time']:
                        fig, ax = plt.subplots(figsize=(10, 4))
                        ride_df['weekday'] = ride_df['datetime'].dt.day_name()
                        ride_df['weekday'].value_counts().reindex([
                            'Monday', 'Tuesday', 'Wednesday',
                            'Thursday', 'Friday', 'Saturday', 'Sunday'
                        ]).plot(kind='bar', ax=ax, color='#1a73e8')
                        ax.set_title('Rides by Weekday')
                        ax.set_ylabel("Number of Rides")
                        plt.tight_layout()
                        plt.show()
                except Exception as e:
                    print(f"⚠️ Time analysis error: {str(e)}")

            # Tab 2: Geospatial Analysis
            with tab2_output:
                try:
                    # Create base map
                    avg_lat = ride_df['pickup_lat'].mean()
                    avg_lng = ride_df['pickup_lng'].mean()
                    m = folium.Map(location=[avg_lat, avg_lng], zoom_start=12)

                    # Add heatmap
                    heat_data = [[row['pickup_lat'], row['pickup_lng']]
                                for _, row in ride_df.iterrows()]
                    plugins.HeatMap(heat_data, radius=15).add_to(m)

                    # Add cluster markers
                    marker_cluster = plugins.MarkerCluster().add_to(m)
                    for _, row in ride_df.sample(min(100, len(ride_df))).iterrows():
                        folium.Marker(
                            [row['pickup_lat'], row['pickup_lng']],
                            popup=f"${row['estimate_price']} {row['ride_type']}",
                            icon=folium.Icon(color='blue' if row['status']=='completed' else 'red')
                        ).add_to(marker_cluster)

                    # Display map
                    display(m)

                    # Price by Area analysis
                    plt.figure(figsize=(10, 4))
                    ride_df['pickup_area'] = pd.cut(ride_df['pickup_lat'], bins=5, labels=[
                        'South', 'Mid-South', 'Central', 'Mid-North', 'North'
                    ])
                    area_prices = ride_df.groupby('pickup_area')['estimate_price'].mean()
                    area_prices.plot(kind='bar', color='#1a73e8')
                    plt.title('Average Price by Area')
                    plt.ylabel("Price ($)")
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"⚠️ Geospatial error: {str(e)}")

            # Tab 3: Driver-Ride Analysis
            with tab3_output:
                try:
                    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 5))

                    # Driver Status
                    driver_df['status'].value_counts().plot(
                        kind='pie', ax=ax1, autopct='%1.1f%%',
                        colors=['#34a853', '#1a73e8', '#fbbc05', '#ea4335']
                    )
                    ax1.set_title('Driver Status Distribution')

                    # Vehicle Type Analysis
                    vehicle_analysis = driver_df.groupby('vehicle_type').agg({
                        'driver_id': 'count',
                        'current_lat': 'mean'  # Just for demo
                    })
                    vehicle_analysis['driver_id'].plot(
                        kind='bar', ax=ax2, color='#1a73e8'
                    )
                    ax2.set_title('Drivers by Vehicle Type')
                    ax2.set_ylabel("Number of Drivers")

                    plt.tight_layout()
                    plt.show()

                    # Driver-Ride Correlation
                    if not ride_df.empty and not driver_df.empty:
                        plt.figure(figsize=(10, 4))
                        hourly_drivers = driver_df.set_index('timestamp').resample('H')['driver_id'].count()
                        hourly_rides = ride_df.set_index('datetime').resample('H')['request_id'].count()
                        pd.DataFrame({
                            'Drivers': hourly_drivers,
                            'Rides': hourly_rides
                        }).plot(title='Drivers vs Rides (Hourly)')
                        plt.ylabel("Count")
                        plt.tight_layout()
                        plt.show()
                except Exception as e:
                    print(f"⚠️ Driver analysis error: {str(e)}")

            # Display tabs
            display(analysis_tabs)

            # ====================
            # Data Preview Section
            # ====================
            display(HTML("<h3 style='margin-top: 20px;'>📋 Data Preview</h3>"))

            # Create tabs for data preview
            preview_tabs = widgets.Tab()
            with widgets.Output() as preview1:
                display(HTML("<h4>Ride Data Sample</h4>"))
                display(ride_df[['datetime', 'ride_type', 'status', 'estimate_price']].head(5))

            with widgets.Output() as preview2:
                display(HTML("<h4>Driver Data Sample</h4>"))
                display(driver_df[['timestamp', 'status', 'vehicle_type']].head(5))

            preview_tabs.children = [preview1, preview2]
            preview_tabs.titles = ['Rides', 'Drivers']
            display(preview_tabs)

        except Exception as e:
            print(f"⚠️ Dashboard error: {str(e)}")

# Set up interactions
refresh_button.on_click(update_dashboard)
status_dropdown.observe(update_dashboard, names='value')
time_window.observe(update_dashboard, names='value')

# Dashboard layout
controls = widgets.HBox([refresh_button, status_dropdown, time_window])
dashboard = widgets.VBox([
    controls,
    output,
    analysis_tabs
])

# Initial display
display(dashboard)
update_dashboard(None)

# Auto-refresh option
import threading
def auto_refresh():
    while True:
        time.sleep(300)  # Refresh every 5 minutes
        refresh_button.click()

threading.Thread(target=auto_refresh, daemon=True).start()